In [190]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import ensemble
from sklearn.metrics import accuracy_score
import sklearn.model_selection as ms

In [209]:
attacks1 = pd.read_csv('/Users/mdrozdov/Documents/watermelon_tree/data/attacks1.csv')

/Users/mdrozdov/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [227]:
attacks1.head()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,443,6,02/03/2018 08:47:38,141385,9,7,553,3773.0,202,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,49684,6,02/03/2018 08:47:38,281,2,1,38,0.0,38,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,443,6,02/03/2018 08:47:40,279824,11,15,1086,10527.0,385,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,443,6,02/03/2018 08:47:40,132,2,0,0,0.0,0,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,443,6,02/03/2018 08:47:41,274016,9,13,1285,6141.0,517,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [243]:
any_inf = {}

In [244]:
for column in attacks1.columns:
    try:
        any_inf[column] = np.isinf(attacks1[column]).sum()
    except:
        print(f'error type in {column}')
        continue

error type in Timestamp
error type in Flow Byts/s
error type in Flow Pkts/s


{'Dst Port': 0,
 'Protocol': 0,
 'Flow Duration': 0,
 'Tot Fwd Pkts': 0,
 'Tot Bwd Pkts': 0,
 'TotLen Fwd Pkts': 0,
 'TotLen Bwd Pkts': 0,
 'Fwd Pkt Len Max': 0,
 'Fwd Pkt Len Min': 0,
 'Fwd Pkt Len Mean': 0,
 'Fwd Pkt Len Std': 0,
 'Bwd Pkt Len Max': 0,
 'Bwd Pkt Len Min': 0,
 'Bwd Pkt Len Mean': 0,
 'Bwd Pkt Len Std': 0,
 'Flow IAT Mean': 0,
 'Flow IAT Std': 0,
 'Flow IAT Max': 0,
 'Flow IAT Min': 0,
 'Fwd IAT Tot': 0,
 'Fwd IAT Mean': 0,
 'Fwd IAT Std': 0,
 'Fwd IAT Max': 0,
 'Fwd IAT Min': 0,
 'Bwd IAT Tot': 0,
 'Bwd IAT Mean': 0,
 'Bwd IAT Std': 0,
 'Bwd IAT Max': 0,
 'Bwd IAT Min': 0,
 'Fwd PSH Flags': 0,
 'Bwd PSH Flags': 0,
 'Fwd URG Flags': 0,
 'Bwd URG Flags': 0,
 'Fwd Header Len': 0,
 'Bwd Header Len': 0,
 'Fwd Pkts/s': 0,
 'Bwd Pkts/s': 0,
 'Pkt Len Min': 0,
 'Pkt Len Max': 0,
 'Pkt Len Mean': 0,
 'Pkt Len Std': 0,
 'Pkt Len Var': 0,
 'FIN Flag Cnt': 0,
 'SYN Flag Cnt': 0,
 'RST Flag Cnt': 0,
 'PSH Flag Cnt': 0,
 'ACK Flag Cnt': 0,
 'URG Flag Cnt': 0,
 'CWE Flag Count': 0,


In [276]:
list(map(lambda x: float(x), attacks1['Flow Byts/s']))

[30597.30523,
 135231.3167,
 41501.0864,
 0.0,
 27100.60726,
 0.0,
 0.0,
 0.0,
 62161.44801,
 203947.3684,
 0.0,
 68.61642698,
 0.0,
 0.0,
 45.75432052,
 59.43639382,
 21.07286398,
 11.40051845,
 923.0324608,
 401041.6667,
 0.0,
 2.998517982,
 0.108957496,
 0.109441951,
 0.0,
 0.0,
 38329.94036,
 153846.1538,
 34233.77452,
 277372.2628,
 31221.64049,
 0.0,
 69092.06957,
 3587.274615,
 10462.04549,
 2206.480084,
 10978.39849,
 394.4855078,
 30869.73008,
 97435.89744,
 83985.76512,
 112855.9464,
 0.0,
 46146.40815,
 24025.38531,
 10319.15476,
 9854.678357,
 27623.60174,
 0.0,
 505.0706434,
 434.3154959,
 22951.58182,
 193750.0,
 9027.267846,
 192727.2727,
 784.7969629,
 0.0,
 945.2775329,
 0.0,
 1071.34206,
 0.0,
 10758.23727,
 409.1741144,
 0.0,
 55.21724914,
 0.0,
 55.21366428,
 0.0,
 0.0,
 0.0,
 2.998474713,
 0.0,
 82.19121377,
 21.26652803,
 46.25083218,
 15500000.0,
 19.76747649,
 19.66285723,
 29.196083,
 19.49527641,
 10.67743426,
 0.0,
 57.12935646,
 0.0,
 0.0,
 3.365457547,
 47.

In [211]:
attacks1 = attacks1.dropna()

In [212]:
attacks1.Label = np.where(attacks1.Label == 'Benign',0,1)

In [213]:
Y = attacks1.Label

In [214]:
X = attacks1.drop(['Label','Timestamp'],axis=1)

In [215]:
x_train, x_test, y_train, y_test = ms.train_test_split(X,Y, test_size = 0.2, shuffle = True)

In [216]:
rf = ensemble.RandomForestClassifier(oob_score = True,
                                    n_jobs = -1,
                                    n_estimators = 1000)

In [217]:
rf.fit(x_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').